In [1]:
import numpy as np
import pandas as pd

from flagger import PLFlagger, PLNNFlagger
from flagger_tools import PLFlaggerTest
from flagger_tools import get_sampler_data
from matplotlib import pyplot as plt

<string>:11: SyntaxWarning: invalid escape sequence '\s'
<string>:193: SyntaxWarning: invalid escape sequence '\s'


### Real World Data

Create sampler from data

In [2]:
r = 0.2
P0 = pd.read_csv("Data/sample_reference_group.csv")
P1 = pd.read_csv("Data/sample_critical_group.csv")
sampler = get_sampler_data(r, P0, P1)

Administration parameters

In [3]:
K = 20   # Number of reviews per admin
M = 3    # Length of feature vector
N = 100  # Number of test takers per admin
T = 100  # Number of admins (time horizon)

Number of simulated rollouts

In [4]:
rollouts = 20

Simulate PL-LR

In [5]:
lr_opt = 0
rate_lr_opt = np.zeros(T + 1)

for lr in 10 ** np.linspace(-5, 1, 61):
    # Initialize flagger and test
    pl_flagger_lr = PLFlagger(K, M, lr=lr, loss="log_loss")
    test_lr = PLFlaggerTest(pl_flagger_lr, sampler, N, t_warmup=10)

    # Initialize result arrays
    total_lr = np.zeros((rollouts, T + 1))
    detected_lr = np.zeros((rollouts, T + 1))

    # Simulate rollouts
    for n in range(rollouts):
        total_lr[n], detected_lr[n] = test_lr.run(T)

    # Calculate average detection rate
    rate_lr = np.mean(detected_lr[:, 1:] / total_lr[:, 1:], axis=0)

    # Update learning rate if final detection rate exceeds previous best
    if rate_lr[-1] > rate_lr_opt[-1]:
        lr_opt = lr
        rate_lr_opt = rate_lr

print(f"PL-LR: Maximum detection rate = {rate_lr_opt[-1]} at learning rate {lr_opt}")

PL-LR: Maximum detection rate = 0.49169091518580615 at learning rate 5.011872336272725


Simulate PL-SV

In [6]:
lr_opt = 0
rate_sv_opt = np.zeros(T + 1)

for lr in 10 ** np.linspace(-5, 1, 61):
    # Initialize flagger and test
    pl_flagger_sv = PLFlagger(K, M, lr=lr, loss="hinge")
    test_sv = PLFlaggerTest(pl_flagger_sv, sampler, N, t_warmup=10)

    # Initialize result arrays
    total_sv = np.zeros((rollouts, T + 1))
    detected_sv = np.zeros((rollouts, T + 1))

    # Simulate rollouts
    for n in range(rollouts):
        total_sv[n], detected_sv[n] = test_sv.run(T)

    # Calculate average detection rate
    rate_sv = np.mean(detected_sv[:, 1:] / total_sv[:, 1:], axis=0)

    # Update learning rate if final detection rate exceeds previous best
    if rate_sv[-1] > rate_sv_opt[-1]:
        lr_opt = lr
        rate_sv_opt = rate_sv

print(f"PL-SV: Maximum detection rate = {rate_sv_opt[-1]} at learning rate {lr_opt}")

PL-SV: Maximum detection rate = 0.4927686210160961 at learning rate 10.0


Simulate PL-NN

In [ ]:
lr_opt = 0
n_hidden_opt = 0
rate_nn_opt = np.zeros(T + 1)

for n_hidden in range(M, 10 * M, int(np.floor(M / 2))):
    for lr in 10 ** np.linspace(-5, 1, 61):
        # Initialize flagger and test
        pl_flagger_nn = PLFlagger(K, M, lr=lr, loss="hinge")
        test_nn = PLFlaggerTest(pl_flagger_nn, sampler, N, t_warmup=10)
    
        # Initialize result arrays
        total_nn = np.zeros((rollouts, T + 1))
        detected_nn = np.zeros((rollouts, T + 1))
    
        # Simulate rollouts
        for n in range(rollouts):
            total_nn[n], detected_nn[n] = test_nn.run(T)
    
        # Calculate average detection rate
        rate_nn = np.mean(detected_nn[:, 1:] / total_nn[:, 1:], axis=0)
    
        # Update learning rate if final detection rate exceeds previous best
        if rate_nn[-1] > rate_nn_opt[-1]:
            lr_opt = lr
            n_hidden_opt = n_hidden
            rate_nn_opt = rate_nn

print(f"PL-NN: Maximum detection rate = {rate_nn_opt[-1]} at learning rate {lr_opt} and hidden layer width {n_hidden_opt}")

Plot average detection rates

In [ ]:
plt.plot(range(1, T+1), rate_lr_opt, label="PL-LR")
plt.plot(range(1, T+1), rate_sv_opt, label="PL-SV")
plt.plot(range(1, T+1), rate_nn_opt, label="PL-NN")
plt.title("Average Detection Rate")
plt.ylabel("Average Detection Rate")
plt.xlabel("Administration")
plt.legend()
plt.grid()